-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# De-Duping Data
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *dest_file*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img src="https://files.training.databricks.com/images/icon_hint_24.png"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
!pip install mlflow

Collecting mlflow
 Downloading mlflow-1.28.0-py3-none-any.whl (17.0 MB)
 | 10 kB 11.3 MB/s eta 0:00:02 | | 20 kB 2.4 MB/s eta 0:00:07 | | 30 kB 1.9 MB/s eta 0:00:09 | | 40 kB 823 kB/s eta 0:00:21 | | 51 kB 834 kB/s eta 0:00:21 |▏ | 61 kB 999 kB/s eta 0:00:17 |▏ | 71 kB 1.1 MB/s eta 0:00:16 |▏ | 81 kB 1.2 MB/s eta 0:00:15 |▏ | 92 kB 1.3 MB/s eta 0:00:14 |▏ | 102 kB 986 kB/s eta 0:00:18 |▏ | 112 kB 986 kB/s eta 0:00:18 |▎ | 122 kB 986 kB/s eta 0:00:18 |▎ | 133 kB 986 kB/s eta 0:00:18 |▎ | 143 kB 986 kB/s eta 0:00:18 |▎ | 153 kB 986 kB/s eta 0:00:18 |▎ | 163 kB 986 kB/s eta 0:00:18 |▎ | 174 kB 986 kB/s eta 0:00:18 |▍ | 184 kB 986 kB/s eta 0:00:18 |▍ | 194 kB 986 kB/s eta 0:00:18 |▍ | 204 kB 986 kB/s eta 0:00:17 |▍ | 215 kB 986 kB/s eta 0:00:17 |▍ | 225 kB 986 kB/s eta 0:00:17 |▍ | 235 kB 986 kB/s eta 0:00:17 |▌ | 245 kB 986 kB/s eta 0:00:17 |▌ | 256 kB 986 kB/s eta 0:00:17 |▌ | 266 kB 986 kB/s eta 0:00:17 |▌ | 276 kB 986 kB/s eta 0:00:17 |▌ | 286 kB 986 kB/s eta 0:00:17 |▋ | 296 kB 986 kB/s eta 0:00:17 |▋ | 307 kB 986 kB/s eta 0:00:17 |▋ | 317 kB 986 kB/s eta 0:00:17 |▋ | 327 kB 986 kB/s eta 0:00:17 |▋ | 337 kB 986 kB/s eta 0:00:17 |▋ | 348 kB 986 kB/s eta 0:00:17 |▊ | 358 kB 986 kB/s eta 0:00:17 |▊ | 368 kB 986 kB/s eta 0:00:17 |▊ | 378 kB 986 kB/s eta 0:00:17 |▊ | 389 kB 986 kB/s eta 0:00:17 |▊ | 399 kB 986 kB/s eta 0:00:17 |▊ | 409 kB 986 kB/s eta 0:00:17 |▉ | 419 kB 986 kB/s eta 0:00:17 |▉ | 430 kB 986 kB/s eta 0:00:17 |▉ | 440 kB 986 kB/s eta 0:00:17 |▉ | 450 kB 986 kB/s eta 0:00:17 |▉ | 460 kB 986 kB/s eta 0:00:17 |▉ | 471 kB 986 kB/s eta 0:00:17 |█ | 481 kB 986 kB/s eta 0:00:17 |█ | 491 kB 986 kB/s eta 0:00:17 |█ | 501 kB 986 kB/s eta 0:00:17 |█ | 512 kB 986 kB/s eta 0:00:17 |█ | 522 kB 986 kB/s eta 0:00:17 |█ | 532 kB 986 kB/s eta 0:00:17 |█ | 542 kB 986 kB/s eta 0:00:17 |█ | 552 kB 986 kB/s eta 0:00:17 |█ | 563 kB 986 kB/s eta 0:00:17 |█ | 573 kB 986 kB/s eta 0:00:17 |█ | 583 kB 986 kB/s eta 0:00:17 |█▏ | 593 kB 986 kB/s eta 0:00:17 |█▏ | 604 kB 986 kB/s eta 0:00:17 |█▏ | 614 kB 986 kB/s eta 0:00:17 |█▏ | 624 kB 986 kB/s eta 0:00:17 |█▏ | 634 kB 986 kB/s eta 0:00:17 |█▏ | 645 kB 986 kB/s eta 0:00:17 |█▎ | 655 kB 986 kB/s eta 0:00:17 |█▎ | 665 kB 986 kB/s eta 0:00:17 |█▎ | 675 kB 986 kB/s eta 0:00:17 |█▎ | 686 kB 986 kB/s eta 0:00:17 |█▎ | 696 kB 986 kB/s eta 0:00:17 |█▎ | 706 kB 986 kB/s eta 0:00:17 |█▍ | 716 kB 986 kB/s eta 0:00:17 |█▍ | 727 kB 986 kB/s eta 0:00:17 |█▍ | 737 kB 986 kB/s eta 0:00:17 |█▍ | 747 kB 986 kB/s eta 0:00:17 |█▍ | 757 kB 986 kB/s eta 0:00:17 |█▌ | 768 kB 986 kB/s eta 0:00:17 |█▌ | 778 kB 986 kB/s eta 0:00:17 |█▌ | 788 kB 986 kB/s eta 0:00:17 |█▌ | 798 kB 986 kB/s eta 0:00:17 |█▌ | 808 kB 986 kB/s eta 0:00:17 |█▌ | 819 kB 986 kB/s eta 0:00:17 |█▋ | 829 kB 986 kB/s eta 0:00:17 |█▋ | 839 kB 986 kB/s eta 0:00:17 |█▋ | 849 kB 986 kB/s eta 0:00:17 |█▋ | 860 kB 986 kB/s eta 0:00:17 |█▋ | 870 kB 986 kB/s eta 0:00:17 |█▋ | 880 kB 986 kB/s eta 0:00:17 |█▊ | 890 kB 986 kB/s eta 0:00:17 |█▊ | 901 kB 986 kB/s eta 0:00:17 |█▊ | 911 kB 986 kB/s eta 0:00:17 |█▊ | 921 kB 986 kB/s eta 0:00:17 |█▊ | 931 kB 986 kB/s eta 0:00:17 |█▊ | 942 kB 986 kB/s eta 0:00:17 |█▉ | 952 kB 986 kB/s eta 0:00:17 |█▉ | 962 kB 986 kB/s eta 0:00:17 |█▉ | 972 kB 986 kB/s eta 0:00:17 |█▉ | 983 kB 986 kB/s eta 0:00:17 |█▉ | 993 kB 986 kB/s eta 0:00:17 |██ | 1.0 MB 986 kB/s eta 0:00:17 |██ | 1.0 MB 986 kB/s eta 0:00:17 |██ | 1.0 MB 986 kB/s eta 0:00:17 |██ | 1.0 MB 986 kB/s eta 0:00:17 |██ | 1.0 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██ | 1.1 MB 986 kB/s eta 0:00:17 |██▏ | 1.1 MB 986 kB/s eta 0:00:17 |██▏ | 1.1 MB 986 kB/s eta 0:00:17 |██▏ | 1.1 MB 986 kB/s eta 0:00:17 |██▏ | 1.2 MB 986 kB/s eta 0:00:17 |██▏ | 1.2 MB 986 kB/s eta 0:00:17 |█

In [0]:
%run "../Includes/Classroom-Setup"

Defining courseware-specific utility methods...

The source for this dataset is
wasbs://courseware@dbacademy.blob.core.windows.net/scalable-machine-learning-with-apache-spark/v01/

Your dataset directory is
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/machine_learning/datasets

Skipping install of existing dataset.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="https://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html?highlight=dataframe#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. **`dbutils.fs.head()`** (or just **`%fs head`**) is a big help here.

In [0]:
# TODO

source_file = f"{datasets_dir}/dataframes/people-with-dups.txt"
dest_file = userhome + "/people.parquet"


# In case it already exists
dbutils.fs.rm(dest_file, True)

Out[18]: True

In [0]:
# ANSWER

# dropDuplicates() will likely introduce a shuffle, so it helps to reduce the number of post-shuffle partitions.
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [0]:
# ANSWER

df = (spark
      .read
      .option("header", "true")
      .option("inferSchema", "true")
      .option("sep", ":")
      .csv(source_file))

df.count()

Out[20]: 103000

In [0]:
# ANSWER
from pyspark.sql.functions import  col, lower, translate

deduped_df = (df
              .select(col("*"),
                      lower(col("firstName")).alias("lcFirstName"),
                      lower(col("lastName")).alias("lcLastName"),
                      lower(col("middleName")).alias("lcMiddleName"),
                      translate(col("ssn"), "-", "").alias("ssnNums")
                      # regexp_replace(col("ssn"), "-", "").alias("ssnNums")
                      # regexp_replace(col("ssn"), """^(\d{3})(\d{2})(\d{4})$""", "$1-$2-$3").alias("ssnNums")
                     )
             .dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "ssnNums", "gender", "birthDate", "salary"])
             .drop("lcFirstName", "lcMiddleName", "lcLastName", "ssnNums")
            )

In [0]:
# ANSWER

# Now we can save the results. We'll also re-read them and count them, just as a final check.
# Just for fun, we'll use the Snappy compression codec. It's not as compact as Gzip, but it's much faster.
deduped_df.write.mode("overwrite").parquet(dest_file)

deduped_df = spark.read.parquet(dest_file)
print(f"Total Records: {deduped_df.count()}")

Total Records: 100000

In [0]:
# ANSWER

display(dbutils.fs.ls(dest_file))

path,name,size,modificationTime
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/_SUCCESS,_SUCCESS,0,1661401876000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/_committed_1240862853632513444,_committed_1240862853632513444,420,1661401875000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/_started_1240862853632513444,_started_1240862853632513444,0,1661401873000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/part-00000-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-69-1-c000.snappy.parquet,part-00000-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-69-1-c000.snappy.parquet,780865,1661401875000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/part-00001-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-70-1-c000.snappy.parquet,part-00001-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-70-1-c000.snappy.parquet,775438,1661401875000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/part-00002-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-71-1-c000.snappy.parquet,part-00002-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-71-1-c000.snappy.parquet,778067,1661401875000
dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/people.parquet/part-00003-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-72-1-c000.snappy.parquet,part-00003-tid-1240862853632513444-7b16f52d-69ef-49cf-90c7-c81727a7add4-72-1-c000.snappy.parquet,769956,1661401875000


##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **dest_file** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
part_files = len(list(filter(lambda f: f.path.endswith(".parquet"), dbutils.fs.ls(dest_file))))

final_df = spark.read.parquet(dest_file)
final_count = final_df.count()

clearYourResults()
validateYourAnswer("01 Parquet File Exists", 1276280174, part_files)
validateYourAnswer("02 Expected 100000 Records", 972882115, final_count)
summarizeYourResults()

01 Parquet File Exists was NOT correct, your answer: 4
02 Expected 100000 Records was correct, your answer: 100000

01 Parquet File Exists:,FAILED,4
02 Expected 100000 Records:,passed,100000


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>